In [80]:
#On importe les package nécessaire à l'exercice.
import pandas as pd
import sqlite3
import tqdm
import random

#1. Read all the data
df = pd.read_csv("/Users/polo11/Downloads/bc.csv")
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [86]:
print("nombre de ligne :", (df.shape[0]))
print("nombre de colonne :", (df.shape[1]))

nombre de ligne : 569
nombre de colonne : 33


In [87]:
# 2: Insert the bc.csv in the sqlite format.
import sqlite3

conn = sqlite3.connect('/Users/polo11/Downloads/bc.db')
c = conn.cursor()





In [88]:
#3. In the same db create a new table called tomatch with two variables id_ and rdm_float.
#On crée la table tomatch
def create_table():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS tomatch(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
        conn.commit()
    except Exception as e:
        print(str(e))
        
create_table()

In [89]:
# 4: In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.
query = """INSERT INTO tomatch(id_,rdm_float) VALUES (?,?)"""
list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000001,2)):
    values = (i, random.uniform(0,1))
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        conn.commit()
        list_of_insertion = []

  0%|          | 9999/4975001 [00:00<00:04, 1071481.20it/s]


IntegrityError: UNIQUE constraint failed: tomatch.id_

In [90]:
# 5: Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.

c.execute("""SELECT avg(rdm_float), count(rdm_float), max(rdm_float), min(rdm_float), sum(rdm_float) FROM tomatch
             INNER JOIN bc ON tomatch.id_=bc.id""")

docs = c.fetchall()
print(docs)

[(7.826773603667237e+16, 243, 9.208189619803999e+18, -9.164256280137096e+18, 1.9019059856911385e+19)]


In [91]:
# 6: How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

c.execute("SELECT COUNT(*) FROM bc WHERE radius_mean > 15")

count_radius_mean_gt_15 = c.fetchone()[0]

print("Radius moyen > 15 :", count_radius_mean_gt_15)

c.execute("SELECT COUNT(*) FROM bc WHERE radius_mean > 15 AND texture_mean > 20")

count_both_gt_15_20 = c.fetchone()[0]

print("Radius moyen > 15 et texture moyenne > 20 :", count_both_gt_15_20)

Radius moyen > 15 : 173
Radius moyen > 15 et texture moyenne > 20 : 106


In [92]:
# 7: For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups
# Exécuter une requête SQL pour calculer la moyenne de la variable souhaitée pour chaque catégorie de diagnostic
c.execute("""
          SELECT
              diagnosis,
              ROUND(AVG(radius_mean), 2) AS avg_radius_mean,  -- Moyenne arrondie de radius_mean avec 2 chiffres après la virgule
              ROUND(AVG(texture_mean), 2) AS avg_texture_mean   -- Moyenne arrondie de texture_mean avec 2 chiffres après la virgule
          FROM
              bc
          GROUP BY
              diagnosis
          """)

# Récupérer les résultats de la requête
results = c.fetchall()

# Afficher les résultats
for row in results:
    print("Diagnostic:", row[0])
    print("moyenne radius_mean:", row[1])
    print("moyenne texture_mean:", row[2])
    print()

# Calculer la différence entre les moyennes des deux groupes
diff_avg_radius_mean = results[0][1] - results[1][1]
diff_avg_texture_mean = results[0][2] - results[1][2]

print("Différence de radius_mean moyen entre les 2 groupes:", diff_avg_radius_mean)
print("Différence de texture_mean moyenne entre les 2 groupes:", diff_avg_texture_mean)

Diagnostic: B
moyenne radius_mean: 12.15
moyenne texture_mean: 17.91

Diagnostic: M
moyenne radius_mean: 17.46
moyenne texture_mean: 21.6

Différence de radius_mean moyen entre les 2 groupes: -5.3100000000000005
Différence de texture_mean moyenne entre les 2 groupes: -3.6900000000000013


In [93]:
# 8. Change the name of the column "diagnosis" to "label"
c.execute("ALTER TABLE bc RENAME COLUMN diagnosis TO label")

conn.commit()

In [94]:
# 9. Add a column named "day" of type DATE.
c.execute("ALTER TABLE bc ADD COLUMN day DATE")

conn.commit()

In [95]:
# 10. Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)
c.execute("ALTER TABLE bc ADD COLUMN area_mean_int INTEGER")
conn.commit()
c.execute("UPDATE bc SET area_mean_int = CAST(area_mean AS INTEGER)")
conn.commit()

OperationalError: duplicate column name: area_mean_int

In [96]:
# Histoire de s'assurer que tout cela a bien fonctionné :
c.execute("PRAGMA table_info(bc)")

columns_info = c.fetchall()

for column_info in columns_info:
    print(column_info)

(0, 'id', 'INTEGER', 0, None, 0)
(1, 'label', 'TEXT', 0, None, 0)
(2, 'radius_mean', 'REAL', 0, None, 0)
(3, 'texture_mean', 'REAL', 0, None, 0)
(4, 'perimeter_mean', 'REAL', 0, None, 0)
(5, 'area_mean', 'REAL', 0, None, 0)
(6, 'smoothness_mean', 'REAL', 0, None, 0)
(7, 'compactness_mean', 'REAL', 0, None, 0)
(8, 'concavity_mean', 'REAL', 0, None, 0)
(9, 'concave points_mean', 'REAL', 0, None, 0)
(10, 'symmetry_mean', 'REAL', 0, None, 0)
(11, 'fractal_dimension_mean', 'REAL', 0, None, 0)
(12, 'radius_se', 'REAL', 0, None, 0)
(13, 'texture_se', 'REAL', 0, None, 0)
(14, 'perimeter_se', 'REAL', 0, None, 0)
(15, 'area_se', 'REAL', 0, None, 0)
(16, 'smoothness_se', 'REAL', 0, None, 0)
(17, 'compactness_se', 'REAL', 0, None, 0)
(18, 'concavity_se', 'REAL', 0, None, 0)
(19, 'concave points_se', 'REAL', 0, None, 0)
(20, 'symmetry_se', 'REAL', 0, None, 0)
(21, 'fractal_dimension_se', 'REAL', 0, None, 0)
(22, 'radius_worst', 'REAL', 0, None, 0)
(23, 'texture_worst', 'REAL', 0, None, 0)
(24, 'per

In [97]:
#11. Create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.
c.execute("""CREATE TABLE IF NOT EXISTS tomatch2(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
conn.commit()

query = """INSERT INTO tomatch2(id_,rdm_float) VALUES (?,random())"""

list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,10)):
    values = (i,)
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        conn.commit()
        list_of_insertion = []

100%|██████████| 995000/995000 [00:01<00:00, 825431.57it/s]


In [98]:
# 12: Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2.
# Exécuter une requête SQL pour effectuer une jointure interne des tables "bc", "tomatch" et "tomatch2" avec les conditions spécifiées
c.execute("""
          SELECT *
          FROM bc
          INNER JOIN tomatch ON bc.id = tomatch.id_
          INNER JOIN tomatch2 ON bc.id = tomatch2.id_
          WHERE bc.radius_mean > 15
          AND tomatch.rdm_float > 0.50
          AND tomatch2.rdm_float > 0.50
          """)

# Récupérer les résultats de la requête
results = c.fetchall()

# Afficher les résultats
for row in results:
    print(row)

(89812, 'M', 23.51, 24.27, 155.1, 1747.0, 0.1069, 0.1283, 0.2308, 0.141, 0.1797, 0.05506, 1.009, 0.9245, 6.462, 164.1, 0.006292, 0.01971, 0.03582, 0.01301, 0.01479, 0.003118, 30.67, 30.73, 202.4, 2906.0, 0.1515, 0.2678, 0.4819, 0.2089, 0.2593, 0.07738, None, 1747, None, 89812, 4.672834584858535e+18, 89812, 1.877013772783616e+18)
(849014, 'M', 19.81, 22.15, 130.0, 1260.0, 0.09831, 0.1027, 0.1479, 0.09498, 0.1582, 0.05395, 0.7582, 1.017, 5.865, 112.4, 0.006494, 0.01893, 0.03391, 0.01521, 0.01356, 0.001997, 27.32, 30.88, 186.8, 2398.0, 0.1512, 0.315, 0.5372, 0.2388, 0.2768, 0.07615, None, 1260, None, 849014, 8.55887145456864e+18, 849014, 2.7055482863915387e+18)
(852552, 'M', 16.65, 21.38, 110.0, 904.6, 0.1121, 0.1457, 0.1525, 0.0917, 0.1995, 0.0633, 0.8068, 0.9017, 5.455, 102.6, 0.006048, 0.01882, 0.02741, 0.0113, 0.01468, 0.002801, 26.46, 31.56, 177.0, 2215.0, 0.1805, 0.3578, 0.4695, 0.2095, 0.3613, 0.09564, None, 904, None, 852552, 1.1400017340516439e+18, 852552, 1.5400851324462612e+18)